In [ ]:
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
import pandas as pd
from glob import glob
import os
import torch

import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

In [ ]:


x1 = ["none","none","none"]
y1 = [0.3,0.32,0.34]

x2 = ["mild", "mild","mild"]
y2 = [0.6,0.76,0.52]

x3 = ["full", "full","full"]
y3 = [0.8,0.98,0.99]

plt.scatter(x1,y1, label='123')
plt.scatter(["none"], statistics.mean(y1), marker='x', c='black')
plt.scatter(x2,y2, label='234')
plt.scatter(["mild"], statistics.mean(y2), marker='x', c='black')
plt.scatter(x3,y3, label='345')
plt.scatter(["full"], statistics.mean(y3), marker='x', c='black')
plt.title('Dice scores with different seeds and augmentation schemes.')
plt.xlabel('Augmentation scheme')
plt.ylabel('Dice score')
plt.savefig('test_fig.pdf')
plt.legend(title='Seed')
plt.show()

plt.boxplot([y1,y2,y3], labels=["none", "mild", "full"], patch_artist=True, boxprops=dict(facecolor='lightblue'))
plt.title('Dice scores with different seeds and augmentation schemes.')
plt.xlabel('Augmentation scheme')
plt.ylabel('Dice score')
plt.savefig('test_fig2.pdf')
plt.show()


In [ ]:
file_path = './experiment_runs/stable_baseline/evaluation/statistics/modified/*bests'
file_names = glob(file_path + '*.json')
file_names.reverse()
base_names = [os.path.basename(fn).split('.')[0] for fn in file_names]

def extract_raw_with_key(file_names, key):
    metrics = []
    for file_name in file_names:
        data = pd.read_json(file_name)
        metric = list(data[key]['raw_data'])
        metrics.append(metric)
    return metrics

dice_scores = extract_raw_with_key(file_names, 'any_blood_dice')

plt.boxplot(x=dice_scores, labels=['none', 'mild', 'full'], patch_artist=True, boxprops=dict(facecolor='lightblue'))
plt.xlabel('Augmentation scheme')
plt.ylabel('Dice score')
save_path = './results/stable_baseline/augmentation_effect'
plt.savefig(f'{save_path}.pdf')
plt.savefig(f'{save_path}.png')
plt.show()

In [ ]:
import tensorboard as tb
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tensorboard.backend.event_processing.event_multiplexer import EventMultiplexer

path = './experiment_runs/mt_cut_mix/DiMsemc_123_fold_0/'
runs = [os.path.basename(run) for run in glob(f'{path}*')]
print(runs)
em = EventMultiplexer().AddRunsFromDirectory(path)
em.Reload()
print(em._accumulators)

for run in runs:
    try:
        scores = em.Scalars(run, 'semi/validation/any_blood_dice')
        scores = [score.value for score in scores]
        sns.lineplot(x=list(range(len(scores))), y=scores)
    except Exception as e:
        print(e)
        
plt.show()

In [ ]:
result_dir = './experiment_runs/'
experiment_names = [
    'stable_mt_seg',
    'stable_baseline',
    'stable_mt_cut_mix',
    'stable_fm_seg'
]

metric_names = {
    'any_blood_dice',
    'any_blood_iou',
    'slice_specificities',
    'slice_sensitivities',
    'pixel_specificities',
    'pixel_sensitivities'
}

model_paths = [
    '../remote_experiment_runs/',
    '../experiment_runs/'
]

rows = []

N_raw_data = 5

for experiment_name in experiment_names:
    stats_dir=os.path.join(result_dir, experiment_name, 'evaluation/statistics/modified/')
    result_files = glob(stats_dir + '*.json')
    
    highest = None
    file_name_best = None
    best_supervised = None

    for i, r_file in enumerate(result_files):
        results = pd.read_json(r_file)
        if highest is None or results['any_blood_dice']['mean'] > highest['any_blood_dice']['mean']:
            highest = results
            file_name_best = r_file
        
        for model_base_path in model_paths:
            try:
                model_path = os.path.join(model_base_path, experiment_name, 'models', results.ID.raw_data[0])
                model = torch.load(model_path)
                print(f'Succesfully loaded model from {model_path}')
            except Exception as e:
                print(e)


    results = pd.read_json(file_name_best)
    for raw_idx in range(N_raw_data):
        row = dict(
            ID=highest.ID.raw_data[0],
            experiment_ID=file_name_best,
            raw_data=None
        )
        for metric_name in metric_names:
            row[metric_name] = results[metric_name].raw_data[raw_idx]
        rows.append(row)

bests = pd.DataFrame(rows)

In [ ]:
bests

In [ ]:
sns.barplot(data=bests, x='ID', y='any_blood_dice', ci="sd")
plt.xticks(rotation=90)
plt.ylim((0.3,0.45))